## Simple gen ai app using langchain

In [1]:
# Load all the keys

import os
from dotenv import load_dotenv
load_dotenv()

# OpenAI key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Langsmith Key for tracking the project
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
# Data Ingation 
# From the website we need to scrape the text data

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [5]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecomm

In [6]:
# split the docs

from langchain_text_splitters import RecursiveCharacterTextSplitter
split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs_split = split.split_documents(docs)
docs_split

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended Read

In [7]:
# Vector Embedding

from langchain_community.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\nayak\AppData\Local\Temp\ipykernel_1396\2286052212.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [8]:
# Vector store

from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs_split,embedding)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

    Because i have a free account in OPEN AI so getting the error as we can't access the Model

In [9]:
# query the data from vectorstore db

query = "What is langsmith"
result = db.asimilarity_search(query)
result[0].page_content

NameError: name 'db' is not defined

    Because i have a free account in OPEN AI so getting the error as we can't access the Model so db is not created or exicuted

In [10]:
# LLM Model

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-o4")

In [11]:
# Retieval chain, Document chain

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question based on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000270AAE94730>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000270AAEE1360>, root_client=<openai.OpenAI object at 0x00000270AAE94C40>, root_async_client=<openai.AsyncOpenAI object at 0x00000270AAE946D0>, model_name='gpt-o4', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, 

In [12]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended retention traces",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.")]
})


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-o4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

    Because i have a free account in OPEN AI so getting the error as we can't access the Model

In [13]:
# Retriver chain

from langchain.chains import create_retrieval_chain
retriver = db.as_retriever()
retriver_chain = create_retrieval_chain(retriver,document_chain)
retriver_chain

NameError: name 'db' is not defined

    Because i have a free account in OPEN AI so getting the error as we can't access the Model so db is not created or exicuted

In [14]:
## Get the responce from the llm

responce = retriver_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended retention traces"})
responce['answer']

NameError: name 'retriver_chain' is not defined

In [15]:
responce, responce['context'],responce['input'],responce['answer']

NameError: name 'responce' is not defined